# Budget

This module is used to help calculate and programatically set your budget in
YNAB.  The reason for this is that I like to determine my budget amounts based
on some more complex logic that YNAB doesn't natively support.  For example, I
like having a mix of percentage based and fixed amount based categories.  The
initial implementation supports a pattern of:
1. Prefixed-variable categories:
    - These categories are a percentage of your raw income.
2. Fixed categories:
    - These categories are a fixed dollar amount from month to month.
3. Postfixed-variable categories:
    - These categories are a percentage of your remaining income after the
      prefixed-variable and fixed categories have been budgeted.

I also like to have certain categories that rollover from month to month.  YNAB
only supports rolling over budget from one month to the next, but that doesn't
make sense for some categories such as `Utilities` or `Groceries`.  This modules
supports the ability to top-off budgets and take the spillover and apply it to
another category. (usually savings of some kind)

In [ ]:
# | default_exp budget

In [ ]:
# | export
import os
import datetime
import requests
import json
import pandas as pd

In [ ]:
# | export
class Budget:
    """A class to represent a YNAB budget for a given month and provides tooling
    to generate and programatically calculate and set budgets"""

    def __init__(self, date=None):
        """Initializes the Budget object with the date of the budget to be fetched. If no date is provided, the current month is used."""
        self.date = date
        self.token = os.environ["YNAB_TOKEN"]
        if self.token == None:
            raise ValueError("YNAB_TOKEN environment variable not set")
        self.fetch_budget()

    def fetch_budget(self):
        """Fetches budget data from YNAB API and stores it in the Budget object"""
        if self.date == None:
            date = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m") + "-01"
        else:
            date = datetime.datetime.strftime(self.date, "%Y-%m") + "-01"
        month = requests.get(
            f"https://api.youneedabudget.com/v1/budgets/last-used/months/{date}?access_token={self.token}"
        )
        month_data = month.json()["data"]["month"]
        self.month = month_data["month"]
        self.income = month_data["income"]
        self.budgeted = month_data["budgeted"]
        self.activity = month_data["activity"]
        self.to_be_budgeted = month_data["to_be_budgeted"]
        self.age_of_money = month_data["age_of_money"]
        self.categories_df = pd.DataFrame(month_data["categories"])

    def generate_budget_template_csv(self, filename="base_budget.csv"):
        """Generates a base budget CSV file for the month"""
        budget_template = pd.DataFrame(
            {
                "category": self.categories_df.query("hidden == False")
                .name.unique()
                .tolist(),
                "fixed": None,
                "pre_fixed": None,
                "rollover": None,
                "amount": None,
            }
        )
        budget_template.to_csv(filename, index=False)
        return budget_template

    def calculate_budget_from_template(
        self, template_path: str, spillover_category: str
    ):
        """Calculate the budget amounts for each category based off of the budget
        template"""
        budget_template = pd.read_csv(template_path)
        if (
            budget_template.query("fixed == False & pre_fixed == False")["amount"].sum()
            != 1
        ):
            raise ValueError(
                "The sum of the amounts for the variable categories must equal 1"
            )
        to_be_budgeted = self.to_be_budgeted
        pre_fixed_df = budget_template.query("pre_fixed == True").copy()
        fixed_df = budget_template.query("fixed == True").copy()
        variable_df = budget_template.query(
            "fixed == False & pre_fixed == False"
        ).copy()
        no_roll_over_categories = budget_template.query(
            "rollover == False"
        ).category.tolist()
        pre_fixed_df["budgeted"] = pre_fixed_df["amount"] * to_be_budgeted
        fixed_df["budgeted"] = fixed_df["amount"] * 1000
        if (
            pre_fixed_df["budgeted"].sum() + fixed_df["budgeted"].sum()
        ) > to_be_budgeted:
            raise ValueError("The fixed and pre-fixed categories exceed to be budgeted")
        remaining_to_be_budgeted = (
            to_be_budgeted - pre_fixed_df["budgeted"].sum() - fixed_df["budgeted"].sum()
        )
        variable_df["budgeted"] = variable_df.amount * remaining_to_be_budgeted
        full_budget = pd.concat([pre_fixed_df, fixed_df, variable_df])
        # Handle no rollover categories
        for category in no_roll_over_categories:
            remaining = self.categories_df.query(f'name == "{category}"').balance.sum()
            adjustment = (
                full_budget.query(f'category == "{category}"')["budgeted"].sum()
                - remaining
            )
            full_budget.loc[full_budget.category == category, "budgeted"] = adjustment
        # round all the budgeted amounts to the nearest cent
        full_budget["budgeted"] = (full_budget["budgeted"] / 1000).round(2)
        # compare the budgeted amount to the rounded to be budgeted amount
        round_to_be_budgeted = round(self.to_be_budgeted / 1000, 2)

        leftover = round_to_be_budgeted - full_budget.budgeted.sum()
        # put the leftover in the spillover category
        full_budget.loc[
            full_budget.category == spillover_category, "budgeted"
        ] += leftover

        if full_budget.budgeted.sum() != round_to_be_budgeted:
            raise ValueError(
                f"The budgeted amount does not equal to be budgeted, you are off by {(round_to_be_budgeted - full_budget.budgeted.sum())/1000}"
            )
        full_budget["budgeted"] = full_budget["budgeted"] * 1000
        self.new_budget = full_budget

    def set_budget(self):
        """Function to take in a dictionary with budget categories and amounts and write them to a month's budget in YNAB"""
        budget_dictionary = dict(
            zip(self.new_budget.category, self.new_budget.budgeted)
        )
        month = requests.get(
            f"https://api.youneedabudget.com/v1/budgets/last-used/months/{self.date}?access_token={self.token}"
        )
        category_balances = pd.DataFrame(
            json.loads(month.content)["data"]["month"]["categories"]
        )
        category_ids = dict(
            zip(category_balances["name"].values, category_balances["id"].values)
        )
        for cat in budget_dictionary.keys():
            category_id = category_ids[cat]
            data = {"category": {"budgeted": int(budget_dictionary[cat])}}
            cat_response = requests.patch(
                f"https://api.youneedabudget.com/v1/budgets/last-used/months/{self.date}/categories/{category_id}?access_token={self.token}",
                json=data,
            )
            if not cat_response.ok:
                raise ValueError(
                    f"There was an error updating the budget for {cat}. The error was {cat_response.content}"
                )
        print("Budget Updated!")

    def zero_out(self):
        """Function to zero out all budget categories"""
        month = requests.get(
            f"https://api.youneedabudget.com/v1/budgets/last-used/months/{self.date}?access_token={self.token}"
        )
        category_balances = pd.DataFrame(
            json.loads(month.content)["data"]["month"]["categories"]
        )
        category_ids = dict(
            zip(category_balances["name"].values, category_balances["id"].values)
        )
        for cat in category_ids.values():
            data = {"category": {"budgeted": 0}}
            cat_response = requests.patch(
                f"https://api.youneedabudget.com/v1/budgets/last-used/months/{self.date}/categories/{cat}?access_token={self.token}",
                json=data,
            )
            if not cat_response.ok:
                print(cat_response.content)
        print("Budget Zeroed Out!")

### Get this month's budget

In [ ]:
budget = Budget(datetime.datetime(2023, 7, 1))

In [ ]:
budget.categories_df.query("hidden == False")[
    ["name", "budgeted", "activity", "balance"]
].head()

,name,budgeted,activity,balance
0,🙎🏻‍♂️Jairus,0,0,718130
1,⛪️Fast Offerings,0,0,512490
2,🏝Vacation,0,0,1669550
3,🏋️Health,0,0,48410
4,🙍🏼‍♀️Sam,0,0,299290


## Budget template explanation
I like to have 2 types of categories, fixed and variable.  Variable is
effectively a percentage of my income, and fixed is a fixed amount.

I also like to have an ordering of variable -> fixed -> variable.  This is good
for things that I want to have a be a percentage of my total income.  I call
these `pre_fixed`.  An example of this is church tithes, or charitable giving.
After I take out the `pre_fixed` money, I then also take out the `fixed` money.
These are usually things that don't change from month to month and are exact
amounts such as a mortgage, rent, or insurance.  After I take out the `fixed`
money, I then take out the `variable` money.  These are things that are more
flexible and can change from month to month.  An example of this is groceries,
gas, or eating out.  I have a rough amount that I want to spend on these
but it's not an exact number.

The last concept that that is somewhat unique are non-rollover categories.
Natively in YNAB, if you don't spend all of your money in a category, it will
rollover to the next month.  This is great for things like savings, insurance,
or personal spending where having the money build up over time is useful,
however, for other things you don't need money to rollover from one month to the
next.  An example of this is groceries.  If you don't spend all of your grocery
money in one month, you don't want to have that money rollover to the next.


## Build budget CSV
This CSV will have a row for each category.  On each row you will need to mark:

- if it should be a `fixed` category (1 for yes, 0 for no)
- if it is a `pre_fixed`
variable category (1 for yes, 0 for no)
- if it is a `rollover` category (1
for yes, 0 for no).  

You will also need to mark the `amount` for each category:
- If it is intended to be a variable category it should be a percentage (i.e. less
than 1)
- If it is a fixed category it should be a dollar amount
- The sum of your categories that are NOT `fixed` and NOT `pre_fixed` should be 1.

In [ ]:
# | hide
budget_file = pd.DataFrame(
    [
        {
            "category": "Utilities",
            "fixed": 1,
            "pre_fixed": 0,
            "rollover": 0,
            "amount": 300,
        },
        {
            "category": "Groceries",
            "fixed": 1,
            "pre_fixed": 0,
            "rollover": 1,
            "amount": 400,
        },
        {"category": "Gas", "fixed": 1, "pre_fixed": 0, "rollover": 1, "amount": 100},
        {
            "category": "Personal Spending",
            "fixed": 0,
            "pre_fixed": 0,
            "rollover": 1,
            "amount": 0.1,
        },
        {
            "category": "Giving",
            "fixed": 0,
            "pre_fixed": 1,
            "rollover": 1,
            "amount": 0.05,
        },
    ]
)

Here's an example of what a budget file might look like:

In [ ]:
budget_file

,category,fixed,pre_fixed,rollover,amount
0,Utilities,1,0,0,300.00
1,Groceries,1,0,1,400.00
2,Gas,1,0,1,100.00
3,Personal Spending,0,0,1,0.10
4,Giving,0,1,1,0.05


You can also use `budget.generate_budget_template_csv()` to generate a template
based on your current budget. Note: You don't have to include every single
category in your budget file.  YNAB sometimes has strange categories that you
don't really want to budget for.

In [ ]:
budget.generate_budget_template_csv().head()

,category,fixed,pre_fixed,rollover,amount
0,🙎🏻‍♂️Jairus,None,None,None,None
1,⛪️Fast Offerings,None,None,None,None
2,🏝Vacation,None,None,None,None
3,🏋️Health,None,None,None,None
4,🙍🏼‍♀️Sam,None,None,None,None


Now we'll use that file to calculate how much money we should have in each
category.


In [ ]:
budget.calculate_budget_from_template(
    template_path="../family_budget.csv",
    spillover_category="📈M1 Finance",
)

Let's see what our new budget looks like!

In [ ]:
display_categories = budget.new_budget.head().category.unique().tolist()
budget.new_budget.head().sort_values("category")

,category,fixed,pre_fixed,rollover,amount,group,budgeted
1,⛪️Fast Offerings,0,1,1,0.01,Church,49560.0
0,⛪️Tithing,0,1,1,0.10,Church,495560.0
2,🎁Giving,0,1,1,0.01,Church,49560.0
5,🎉Fun Money,1,0,1,250.00,Quality of Life,250000.0
4,🏋️Health,1,0,1,20.00,Quality of Life,20000.0


Now that we have the budget amounts, we'll update that in YNAB using their API.

In [ ]:
budget.set_budget()

Budget Updated!


Let's go get the see what the budget from YNAB looks like.

In [ ]:
budget.fetch_budget()

budget.categories_df.query(f"hidden == False and name in {display_categories}")[
    ["name", "budgeted", "activity", "balance"]
].head().sort_values("name")

,name,budgeted,activity,balance
1,⛪️Fast Offerings,49560,0,562050
28,⛪️Tithing,495560,0,3025030
9,🎁Giving,49560,0,799430
19,🎉Fun Money,250000,0,361790
3,🏋️Health,20000,0,68410


And make sure we don't have any money that hasn't been budgeted.

In [ ]:
print(f"To be budgeted: ${budget.to_be_budgeted/1000:,.2f}")

To be budgeted: $0.00


Clean up!  Since this was just a tutorial we can reset the budget

In [ ]:
budget.zero_out()

Budget Zeroed Out!
